In [6]:
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
import os
import io
import hashlib
import time
from PIL import Image
DRIVER_PATH = r"C:\Users\adity\Desktop\chromeDriver\chromedriver"

In [7]:
def GetImageUrls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    #search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"
    search_url = "https://www.google.com/search?q={q}&tbm=isch&hl=en&tbs=isz:l&sa=X&ved=0CAIQpwVqFwoTCPigu4idg_MCFQAAAAAdAAAAABAJ&biw=1519&bih=754"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [8]:
def DownloadImages(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=95)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [9]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=60):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = GetImageUrls(search_term, number_images, wd=wd, sleep_between_interactions=1)
        
    for elem in res:
        DownloadImages(target_folder,elem)

In [10]:
search_term = "Turmericroot"
search_and_download(search_term = search_term,driver_path = DRIVER_PATH, number_images = 300)

Found: 100 search results. Extracting links from 0:100
Found: 159 image links, looking for more ...
Found: 312 search results. Extracting links from 100:312
Found: 301 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS0Km2sURCXDYdLnSOhhC4uny6Sg1CpmqETIw&usqp=CAU - as ./images\turmericroot\6297bea94f.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRiAyTPxuqx7tW7y_Hwgr2XiZ_aItDkuSN2sg&usqp=CAU - as ./images\turmericroot\9048f44742.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSngZkpzWQdgpuKR0KcOj8XTm3s1SDKqvkRrg&usqp=CAU - as ./images\turmericroot\fe69b3872d.jpg
SUCCESS - saved https://images.squarespace-cdn.com/content/v1/574b10348259b5de6732578e/1580156401140-CPOE57X7XMHSBO3JDEKI/Screenshot%2B2020-01-27%2Bat%2B20.17.45.jpg - as ./images\turmericroot\eb0ee79a78.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQktje4WeO1F4irIf2sFo6NQq5s2bpM_YSEnQ&usqp=CAU - as ./images\t